In [1]:
import numpy as np
import os
from keras.layers import Conv3D, MaxPooling3D
from keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential
from keras.layers import Activation, ZeroPadding3D, TimeDistributed, LSTM, GRU, Reshape
#from keras.utils import plot_model
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import gc
from sklearn.metrics import classification_report, balanced_accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from itertools import cycle
from sklearn.preprocessing import LabelEncoder
import sklearn
import seaborn as sns
import tensorflow as tf
from keras import regularizers
from tensorflow.keras.utils import to_categorical
from prettytable import PrettyTable
from matplotlib import pyplot as plt

import time

# noise reduce

In [3]:
import cv2
import os

def reduce_noise(input_folder, output_folder):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop through each file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".mp4"):  # Assuming all files are videos
            video_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)

            # Open the video file
            cap = cv2.VideoCapture(video_path)
            frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            fps = int(cap.get(cv2.CAP_PROP_FPS))

            # Create VideoWriter object
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

            # Process each frame
            for _ in range(frame_count):
                ret, frame = cap.read()
                if ret:
                    # Apply Gaussian blur for noise reduction
                    blurred_frame = cv2.GaussianBlur(frame, (5, 5), 0)
                    out.write(blurred_frame)

            # Release resources
            cap.release()
            out.release()

            print(f"Noise reduction completed for {filename}")

# Example usage:
input_folder = "C:\\Users\\sumit\\Downloads\\New Female Dataset"
output_folder = "C:\\Users\\sumit\\Downloads\\Noise reduction 1\\output"
reduce_noise(input_folder, output_folder)


Noise reduction completed for 0.mp4
Noise reduction completed for 1.mp4
Noise reduction completed for 10.mp4
Noise reduction completed for 2.mp4
Noise reduction completed for 3.mp4
Noise reduction completed for 4.mp4
Noise reduction completed for 5.mp4
Noise reduction completed for 6.mp4
Noise reduction completed for 7.mp4
Noise reduction completed for 8.mp4
Noise reduction completed for 9.mp4
Noise reduction completed for box.mp4
Noise reduction completed for fun.mp4
Noise reduction completed for good.mp4
Noise reduction completed for grow.mp4
Noise reduction completed for hero.mp4
Noise reduction completed for trust.mp4


# convert to frames

In [4]:
import os
import cv2

def extract_frames(video_folder, output_folder, num_frames=8):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate through each video file in the folder
    for filename in os.listdir(video_folder):
        if filename.endswith(".mp4") or filename.endswith(".avi"):
            video_path = os.path.join(video_folder, filename)
            video_name = os.path.splitext(filename)[0]

            # Create a subfolder for each video
            video_output_folder = os.path.join(output_folder, video_name)
            if not os.path.exists(video_output_folder):
                os.makedirs(video_output_folder)

            # Open the video file
            cap = cv2.VideoCapture(video_path)

            # Get the total number of frames in the video
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

            # Calculate the frame interval to get exactly num_frames frames
            frame_interval = max(1, total_frames // num_frames)

            frame_count = 0

            # Read and process each frame
            while(cap.isOpened() and frame_count < num_frames):
                ret, frame = cap.read()
                if not ret:
                    break

                # Save the frame
                output_frame_path = os.path.join(video_output_folder, f"{video_name}_frame_{frame_count}.jpg")
                cv2.imwrite(output_frame_path, frame)

                frame_count += 1

                # Move to the next frame based on the calculated interval
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_count * frame_interval)

            cap.release()

# Example usage:
video_dataset_folder = "C:\\Users\\sumit\\Downloads\\Noise reduction 1\\output"
output_frames_folder = "C:\\Users\\sumit\\Downloads\\Noise reduction 1\\Frames"
extract_frames(video_dataset_folder, output_frames_folder, num_frames=8)


# convert to glcm frames

In [5]:
import os
import cv2
import numpy as np
from skimage.feature import graycomatrix, graycoprops

def glcm_noise_reduction(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Compute the GLCM of the grayscale image
    glcm = graycomatrix(gray, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4])

    # Compute the mean of the GLCM
    mean_glcm = np.mean(glcm)

    # Subtract the mean from the original image to reduce noise
    reduced_noise_image = gray - mean_glcm

    return reduced_noise_image

# Specify the directory containing the folders with frames
root_dir = "C:\\Users\\sumit\\Downloads\\Noise reduction 1\\Frames"

# Create a new folder to save all the noise-reduced images
new_root_dir = "C:\\Users\\sumit\\Downloads\\Noise reduction 1\\output_denoised_image_folder"

# Loop over each folder (video)
for folder in os.listdir(root_dir):
    folder_path = os.path.join(root_dir, folder)
    new_folder_path = os.path.join(new_root_dir, folder)
    os.makedirs(new_folder_path, exist_ok=True)

    # Loop over each image (frame) in the folder
    for frame in os.listdir(folder_path):
        frame_path = os.path.join(folder_path, frame)

        # Read the image
        image = cv2.imread(frame_path)

        # Apply GLCM noise reduction
        reduced_noise_image = glcm_noise_reduction(image)

        # Save the noise-reduced image in the new folder
        cv2.imwrite(os.path.join(new_folder_path, frame), reduced_noise_image)

print('Noise reduction completed.')


Noise reduction completed.


# validation ResnetMobile

In [14]:
from tensorflow.keras.models import load_model

model_paths = [
    #['D:\\FINAL MAJOR20-2\\ALL MODELS 2\\VGG\\VGG_kfold(1)_1.h5', 'D:\\FINAL MAJOR20-2\\ALL MODELS 2\\VGG\\VGG_kfold(1)_2.h5',
    # 'D:\\FINAL MAJOR20-2\\ALL MODELS 2\\VGG\\VGG_kfold(1)_3.h5', 'D:\\FINAL MAJOR20-2\\ALL MODELS 2\\VGG\\VGG_kfold(1)_4.h5',
    # 'D:\\FINAL MAJOR20-2\\ALL MODELS 2\\VGG\\VGG_kfold(1)_5.h5'],

    #['D:\\FINAL MAJOR20-2\\ALL MODELS\\CNN_RNN\\cnn-rnn_kfold(1)_1.h5', 'D:\\FINAL MAJOR20-2\\ALL MODELS\\CNN_RNN\\cnn-rnn_kfold(1)_2.h5',
    # 'D:\\FINAL MAJOR20-2\\ALL MODELS\\CNN_RNN\\cnn-rnn_kfold(1)_3.h5', 'D:\\FINAL MAJOR20-2\\ALL MODELS\\CNN_RNN\\cnn-rnn_kfold(1)_4.h5',
    # 'D:\\FINAL MAJOR20-2\\ALL MODELS\\CNN_RNN\\cnn-rnn_kfold(1)_4.h5'],

    ['D:\\FINAL MAJOR20-2\\ALL MODELS 2\\ResnetMobile\\resnetMobile_kfold(1)_1.h5', 'D:\\FINAL MAJOR20-2\\ALL MODELS 2\\ResnetMobile\\resnetMobile_kfold(1)_2.h5',
     'D:\\FINAL MAJOR20-2\\ALL MODELS 2\\ResnetMobile\\resnetMobile_kfold(1)_3.h5', 'D:\\FINAL MAJOR20-2\\ALL MODELS 2\\ResnetMobile\\resnetMobile_kfold(1)_4.h5',
     'D:\\FINAL MAJOR20-2\\ALL MODELS 2\\ResnetMobile\\resnetMobile_kfold(1)_5.h5'],
    # Add paths for other models similarly
]

models = []
for model_fold_paths in model_paths:
    model_fold = []
    for model_path in model_fold_paths:
        model = load_model(model_path)
        model_fold.append(model)
    models.append(model_fold)
#model2 = load_model('D:\\FINAL MAJOR20-2\\VGG16\\vgg.h5')
#model3 = load_model('D:\\FINAL MAJOR20-2\\CNN_RNN\\cnn_rnn100.h5')
#model4 = load_model('D:\\FINAL MAJOR20-2\\resnet\\resnet.h5')
#model5 = load_model('D:\\FINAL MAJOR20-2\\ResnetMobile\\resnetMobile.h5')
#model6 = load_model('D:\\FINAL MAJOR20-2\\ResnetMobile_VGG\\resnetMobile_vgg_combined.h5')
#model7 = load_model('D:\\FINAL MAJOR20-2\\VGG-lstm\\vgg-lstm.h5')
#model8 = load_model('D:\\FINAL MAJOR20-2\\GLCM-LSTM\\kaggleDataset\\before_Kfold\\GLCM_LSTM.h5')

In [15]:

# Define constants
IMAGE_SIZE = (96, 72)
BATCH_SIZE = 16
EPOCHS = 100
NUM_CLASSES = 18
TRAIN_DATA_DIR = "C:\\Users\\sumit\\Downloads\\Noise reduction 1\\output_denoised_image_folder"

# Create a new data generator for the test data
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Generate the test data generator
test_generator = test_datagen.flow_from_directory(
    directory="C:\\Users\\sumit\\Downloads\\Noise reduction 1\\output_denoised_image_folder",  # Specify the directory argument
    target_size=IMAGE_SIZE,
    batch_size=1,  # Use batch size 1 for single image prediction
    class_mode='categorical',
    shuffle=False  # Keep the order of predictions
)


# Get the number of samples in the test set
num_test_samples = test_generator.samples

# Get the predictions for the test set
y_pred = model.predict(test_generator, steps=num_test_samples)

# Get the ground truth labels for the test set
y_true = test_generator.classes

# Convert predictions to class labels
y_pred_classes = np.argmax(y_pred, axis=1)

# Get class labels from the generator
class_labels = list(test_generator.class_indices.keys())

# Convert class labels to numeric labels
numeric_labels = [test_generator.class_indices[label] for label in class_labels]

# Map numeric labels to class labels
numeric_to_label = dict(zip(numeric_labels, class_labels))

# Map numeric predictions to class labels
y_pred_labels = [numeric_to_label[label] for label in y_pred_classes]

# Convert ground truth labels to class labels
y_true_labels = [numeric_to_label[label] for label in y_true]

# Display the class labels and their numeric mappings
print("Class Labels:")
print(class_labels)
print("Numeric Labels:")
print(numeric_labels)

# Display the true and predicted labels for the test set
print("True Labels:")
print(y_true_labels)
print("Predicted Labels:")
print(y_pred_labels)

# Calculate the accuracy on the test set
test_acc = np.sum(y_pred_classes == y_true) / num_test_samples
print(f'Test Accuracy: {test_acc * 100}')


Found 136 images belonging to 17 classes.
Class Labels:
['0', '1', '10', '2', '3', '4', '5', '6', '7', '8', '9', 'box', 'fun', 'good', 'grow', 'hero', 'trust']
Numeric Labels:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
True Labels:
['0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '1', '1', '1', '1', '1', '1', '10', '10', '10', '10', '10', '10', '10', '10', '2', '2', '2', '2', '2', '2', '2', '2', '3', '3', '3', '3', '3', '3', '3', '3', '4', '4', '4', '4', '4', '4', '4', '4', '5', '5', '5', '5', '5', '5', '5', '5', '6', '6', '6', '6', '6', '6', '6', '6', '7', '7', '7', '7', '7', '7', '7', '7', '8', '8', '8', '8', '8', '8', '8', '8', '9', '9', '9', '9', '9', '9', '9', '9', 'box', 'box', 'box', 'box', 'box', 'box', 'box', 'box', 'fun', 'fun', 'fun', 'fun', 'fun', 'fun', 'fun', 'fun', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'grow', 'grow', 'grow', 'grow', 'grow', 'grow', 'grow', 'grow', 'hero', 'hero', 'hero', 'hero', 'hero', 'hero', 'hero', 'hero',

# CNN

In [10]:
from tensorflow.keras.models import load_model

model_paths = [
    #['D:\\FINAL MAJOR20-2\\ALL MODELS 2\\VGG\\VGG_kfold(1)_1.h5', 'D:\\FINAL MAJOR20-2\\ALL MODELS 2\\VGG\\VGG_kfold(1)_2.h5',
    # 'D:\\FINAL MAJOR20-2\\ALL MODELS 2\\VGG\\VGG_kfold(1)_3.h5', 'D:\\FINAL MAJOR20-2\\ALL MODELS 2\\VGG\\VGG_kfold(1)_4.h5',
    # 'D:\\FINAL MAJOR20-2\\ALL MODELS 2\\VGG\\VGG_kfold(1)_5.h5'],

    #['D:\\FINAL MAJOR20-2\\ALL MODELS\\CNN_RNN\\cnn-rnn_kfold(1)_1.h5', 'D:\\FINAL MAJOR20-2\\ALL MODELS\\CNN_RNN\\cnn-rnn_kfold(1)_2.h5',
    # 'D:\\FINAL MAJOR20-2\\ALL MODELS\\CNN_RNN\\cnn-rnn_kfold(1)_3.h5', 'D:\\FINAL MAJOR20-2\\ALL MODELS\\CNN_RNN\\cnn-rnn_kfold(1)_4.h5',
    # 'D:\\FINAL MAJOR20-2\\ALL MODELS\\CNN_RNN\\cnn-rnn_kfold(1)_4.h5'],

    ['D:\\FINAL MAJOR20-2\\CrossValidation\\CNN\\own\\cnn_kfold(1)_1.h5', 'D:\\FINAL MAJOR20-2\\CrossValidation\\CNN\\own\\cnn_kfold(1)_2.h5',
     'D:\\FINAL MAJOR20-2\\CrossValidation\\CNN\\own\\cnn_kfold(1)_3.h5', 'D:\\FINAL MAJOR20-2\\CrossValidation\\CNN\\own\\cnn_kfold(1)_4.h5',
     'D:\\FINAL MAJOR20-2\\CrossValidation\\CNN\\own\\cnn_kfold(1)_5.h5'],
    # Add paths for other models similarly
]

models = []
for model_fold_paths in model_paths:
    model_fold = []
    for model_path in model_fold_paths:
        model = load_model(model_path)
        model_fold.append(model)
    models.append(model_fold)
#model2 = load_model('D:\\FINAL MAJOR20-2\\VGG16\\vgg.h5')
#model3 = load_model('D:\\FINAL MAJOR20-2\\CNN_RNN\\cnn_rnn100.h5')
#model4 = load_model('D:\\FINAL MAJOR20-2\\resnet\\resnet.h5')
#model5 = load_model('D:\\FINAL MAJOR20-2\\ResnetMobile\\resnetMobile.h5')
#model6 = load_model('D:\\FINAL MAJOR20-2\\ResnetMobile_VGG\\resnetMobile_vgg_combined.h5')
#model7 = load_model('D:\\FINAL MAJOR20-2\\VGG-lstm\\vgg-lstm.h5')
#model8 = load_model('D:\\FINAL MAJOR20-2\\GLCM-LSTM\\kaggleDataset\\before_Kfold\\GLCM_LSTM.h5')

In [13]:

# Define constants
IMAGE_SIZE = (96, 72)
BATCH_SIZE = 16
EPOCHS = 100
NUM_CLASSES = 18
TRAIN_DATA_DIR = "C:\\Users\\sumit\\Downloads\\Noise reduction 1\\output_denoised_image_folder"

# Create a new data generator for the test data
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Generate the test data generator
test_generator = test_datagen.flow_from_directory(
    directory="C:\\Users\\sumit\\Downloads\\Noise reduction 1\\output_denoised_image_folder",  # Specify the directory argument
    target_size=IMAGE_SIZE,
    batch_size=1,  # Use batch size 1 for single image prediction
    class_mode='categorical',
    shuffle=False  # Keep the order of predictions
)


# Get the number of samples in the test set
num_test_samples = test_generator.samples

# Get the predictions for the test set
y_pred = model.predict(test_generator, steps=num_test_samples)

# Get the ground truth labels for the test set
y_true = test_generator.classes

# Convert predictions to class labels
y_pred_classes = np.argmax(y_pred, axis=1)

# Get class labels from the generator
class_labels = list(test_generator.class_indices.keys())

# Convert class labels to numeric labels
numeric_labels = [test_generator.class_indices[label] for label in class_labels]

# Map numeric labels to class labels
numeric_to_label = dict(zip(numeric_labels, class_labels))

# Map numeric predictions to class labels
y_pred_labels = [numeric_to_label[label] for label in y_pred_classes]

# Convert ground truth labels to class labels
y_true_labels = [numeric_to_label[label] for label in y_true]

# Display the class labels and their numeric mappings
print("Class Labels:")
print(class_labels)
print("Numeric Labels:")
print(numeric_labels)

# Display the true and predicted labels for the test set
print("True Labels:")
print(y_true_labels)
print("Predicted Labels:")
print(y_pred_labels)

# Calculate the accuracy on the test set
test_acc = np.sum(y_pred_classes == y_true) / num_test_samples
print(f'Test Accuracy: {test_acc * 100}')


Found 136 images belonging to 17 classes.
Class Labels:
['0', '1', '10', '2', '3', '4', '5', '6', '7', '8', '9', 'box', 'fun', 'good', 'grow', 'hero', 'trust']
Numeric Labels:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
True Labels:
['0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '1', '1', '1', '1', '1', '1', '10', '10', '10', '10', '10', '10', '10', '10', '2', '2', '2', '2', '2', '2', '2', '2', '3', '3', '3', '3', '3', '3', '3', '3', '4', '4', '4', '4', '4', '4', '4', '4', '5', '5', '5', '5', '5', '5', '5', '5', '6', '6', '6', '6', '6', '6', '6', '6', '7', '7', '7', '7', '7', '7', '7', '7', '8', '8', '8', '8', '8', '8', '8', '8', '9', '9', '9', '9', '9', '9', '9', '9', 'box', 'box', 'box', 'box', 'box', 'box', 'box', 'box', 'fun', 'fun', 'fun', 'fun', 'fun', 'fun', 'fun', 'fun', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'grow', 'grow', 'grow', 'grow', 'grow', 'grow', 'grow', 'grow', 'hero', 'hero', 'hero', 'hero', 'hero', 'hero', 'hero', 'hero',

# Resnet

In [ ]:
from tensorflow.keras.models import load_model

model_paths = [
    #['D:\\FINAL MAJOR20-2\\ALL MODELS 2\\VGG\\VGG_kfold(1)_1.h5', 'D:\\FINAL MAJOR20-2\\ALL MODELS 2\\VGG\\VGG_kfold(1)_2.h5',
    # 'D:\\FINAL MAJOR20-2\\ALL MODELS 2\\VGG\\VGG_kfold(1)_3.h5', 'D:\\FINAL MAJOR20-2\\ALL MODELS 2\\VGG\\VGG_kfold(1)_4.h5',
    # 'D:\\FINAL MAJOR20-2\\ALL MODELS 2\\VGG\\VGG_kfold(1)_5.h5'],

    #['D:\\FINAL MAJOR20-2\\ALL MODELS\\CNN_RNN\\cnn-rnn_kfold(1)_1.h5', 'D:\\FINAL MAJOR20-2\\ALL MODELS\\CNN_RNN\\cnn-rnn_kfold(1)_2.h5',
    # 'D:\\FINAL MAJOR20-2\\ALL MODELS\\CNN_RNN\\cnn-rnn_kfold(1)_3.h5', 'D:\\FINAL MAJOR20-2\\ALL MODELS\\CNN_RNN\\cnn-rnn_kfold(1)_4.h5',
    # 'D:\\FINAL MAJOR20-2\\ALL MODELS\\CNN_RNN\\cnn-rnn_kfold(1)_4.h5'],

    ['D:\\FINAL MAJOR20-2\\CrossValidation\\Resnet\\own\\resnet_kfold(1)_1.h5','D:\\FINAL MAJOR20-2\\CrossValidation\\Resnet\\own\\resnet_kfold(1)_2.h5',
     'D:\\FINAL MAJOR20-2\\CrossValidation\\Resnet\\own\\resnet_kfold(1)_3.h5','D:\\FINAL MAJOR20-2\\CrossValidation\\Resnet\\own\\resnet_kfold(1)_4.h5',
     'D:\\FINAL MAJOR20-2\\CrossValidation\\Resnet\\own\\resnet_kfold(1)_5.h5'],
    # Add paths for other models similarly
]

models = []
for model_fold_paths in model_paths:
    model_fold = []
    for model_path in model_fold_paths:
        model = load_model(model_path)
        model_fold.append(model)
    models.append(model_fold)
#model2 = load_model('D:\\FINAL MAJOR20-2\\VGG16\\vgg.h5')
#model3 = load_model('D:\\FINAL MAJOR20-2\\CNN_RNN\\cnn_rnn100.h5')
#model4 = load_model('D:\\FINAL MAJOR20-2\\resnet\\resnet.h5')
#model5 = load_model('D:\\FINAL MAJOR20-2\\ResnetMobile\\resnetMobile.h5')
#model6 = load_model('D:\\FINAL MAJOR20-2\\ResnetMobile_VGG\\resnetMobile_vgg_combined.h5')
#model7 = load_model('D:\\FINAL MAJOR20-2\\VGG-lstm\\vgg-lstm.h5')
#model8 = load_model('D:\\FINAL MAJOR20-2\\GLCM-LSTM\\kaggleDataset\\before_Kfold\\GLCM_LSTM.h5')

In [ ]:

# Define constants
IMAGE_SIZE = (96, 72)
BATCH_SIZE = 16
EPOCHS = 100
NUM_CLASSES = 18
TRAIN_DATA_DIR = "C:\\Users\\sumit\\Downloads\\Noise reduction 1\\output_denoised_image_folder"

# Create a new data generator for the test data
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Generate the test data generator
test_generator = test_datagen.flow_from_directory(
    directory="C:\\Users\\sumit\\Downloads\\Noise reduction 1\\output_denoised_image_folder",  # Specify the directory argument
    target_size=IMAGE_SIZE,
    batch_size=1,  # Use batch size 1 for single image prediction
    class_mode='categorical',
    shuffle=False  # Keep the order of predictions
)


# Get the number of samples in the test set
num_test_samples = test_generator.samples

# Get the predictions for the test set
y_pred = model.predict(test_generator, steps=num_test_samples)

# Get the ground truth labels for the test set
y_true = test_generator.classes

# Convert predictions to class labels
y_pred_classes = np.argmax(y_pred, axis=1)

# Get class labels from the generator
class_labels = list(test_generator.class_indices.keys())

# Convert class labels to numeric labels
numeric_labels = [test_generator.class_indices[label] for label in class_labels]

# Map numeric labels to class labels
numeric_to_label = dict(zip(numeric_labels, class_labels))

# Map numeric predictions to class labels
y_pred_labels = [numeric_to_label[label] for label in y_pred_classes]

# Convert ground truth labels to class labels
y_true_labels = [numeric_to_label[label] for label in y_true]

# Display the class labels and their numeric mappings
print("Class Labels:")
print(class_labels)
print("Numeric Labels:")
print(numeric_labels)

# Display the true and predicted labels for the test set
print("True Labels:")
print(y_true_labels)
print("Predicted Labels:")
print(y_pred_labels)

# Calculate the accuracy on the test set
test_acc = np.sum(y_pred_classes == y_true) / num_test_samples
print(f'Test Accuracy: {test_acc * 100}')


Found 136 images belonging to 17 classes.
Class Labels:
['0', '1', '10', '2', '3', '4', '5', '6', '7', '8', '9', 'box', 'fun', 'good', 'grow', 'hero', 'trust']
Numeric Labels:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
True Labels:
['0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '1', '1', '1', '1', '1', '1', '10', '10', '10', '10', '10', '10', '10', '10', '2', '2', '2', '2', '2', '2', '2', '2', '3', '3', '3', '3', '3', '3', '3', '3', '4', '4', '4', '4', '4', '4', '4', '4', '5', '5', '5', '5', '5', '5', '5', '5', '6', '6', '6', '6', '6', '6', '6', '6', '7', '7', '7', '7', '7', '7', '7', '7', '8', '8', '8', '8', '8', '8', '8', '8', '9', '9', '9', '9', '9', '9', '9', '9', 'box', 'box', 'box', 'box', 'box', 'box', 'box', 'box', 'fun', 'fun', 'fun', 'fun', 'fun', 'fun', 'fun', 'fun', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'grow', 'grow', 'grow', 'grow', 'grow', 'grow', 'grow', 'grow', 'hero', 'hero', 'hero', 'hero', 'hero', 'hero', 'hero', 'hero',

# ResnetMobile_VGG

In [6]:
from tensorflow.keras.models import load_model

model_paths = [
    #['D:\\FINAL MAJOR20-2\\ALL MODELS 2\\VGG\\VGG_kfold(1)_1.h5', 'D:\\FINAL MAJOR20-2\\ALL MODELS 2\\VGG\\VGG_kfold(1)_2.h5',
    # 'D:\\FINAL MAJOR20-2\\ALL MODELS 2\\VGG\\VGG_kfold(1)_3.h5', 'D:\\FINAL MAJOR20-2\\ALL MODELS 2\\VGG\\VGG_kfold(1)_4.h5',
    # 'D:\\FINAL MAJOR20-2\\ALL MODELS 2\\VGG\\VGG_kfold(1)_5.h5'],

    #['D:\\FINAL MAJOR20-2\\ALL MODELS\\CNN_RNN\\cnn-rnn_kfold(1)_1.h5', 'D:\\FINAL MAJOR20-2\\ALL MODELS\\CNN_RNN\\cnn-rnn_kfold(1)_2.h5',
    # 'D:\\FINAL MAJOR20-2\\ALL MODELS\\CNN_RNN\\cnn-rnn_kfold(1)_3.h5', 'D:\\FINAL MAJOR20-2\\ALL MODELS\\CNN_RNN\\cnn-rnn_kfold(1)_4.h5',
    # 'D:\\FINAL MAJOR20-2\\ALL MODELS\\CNN_RNN\\cnn-rnn_kfold(1)_4.h5'],

    ['D:\\FINAL MAJOR20-2\\ALL MODELS 2\\ResnetMobileVGG\\resnetMobile-vgg_kfold(1)_1.h5', 'D:\\FINAL MAJOR20-2\\ALL MODELS 2\\ResnetMobileVGG\\resnetMobile-vgg_kfold(1)_2.h5',
     'D:\\FINAL MAJOR20-2\\ALL MODELS 2\\ResnetMobileVGG\\resnetMobile-vgg_kfold(1)_3.h5', 'D:\\FINAL MAJOR20-2\\ALL MODELS 2\\ResnetMobileVGG\\resnetMobile-vgg_kfold(1)_4.h5',
     'D:\\FINAL MAJOR20-2\\ALL MODELS 2\\ResnetMobileVGG\\resnetMobile-vgg_kfold(1)_5.h5'],
    # Add paths for other models similarly
]

models = []
for model_fold_paths in model_paths:
    model_fold = []
    for model_path in model_fold_paths:
        model = load_model(model_path)
        model_fold.append(model)
    models.append(model_fold)
#model2 = load_model('D:\\FINAL MAJOR20-2\\VGG16\\vgg.h5')
#model3 = load_model('D:\\FINAL MAJOR20-2\\CNN_RNN\\cnn_rnn100.h5')
#model4 = load_model('D:\\FINAL MAJOR20-2\\resnet\\resnet.h5')
#model5 = load_model('D:\\FINAL MAJOR20-2\\ResnetMobile\\resnetMobile.h5')
#model6 = load_model('D:\\FINAL MAJOR20-2\\ResnetMobile_VGG\\resnetMobile_vgg_combined.h5')
#model7 = load_model('D:\\FINAL MAJOR20-2\\VGG-lstm\\vgg-lstm.h5')
#model8 = load_model('D:\\FINAL MAJOR20-2\\GLCM-LSTM\\kaggleDataset\\before_Kfold\\GLCM_LSTM.h5')

In [8]:

# Define constants
IMAGE_SIZE = (96, 72)
BATCH_SIZE = 16
EPOCHS = 100
NUM_CLASSES = 18
TRAIN_DATA_DIR = "C:\\Users\\sumit\\Downloads\\Noise reduction 1\\output_denoised_image_folder"

# Create a new data generator for the test data
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Generate the test data generator
test_generator = test_datagen.flow_from_directory(
    directory="C:\\Users\\sumit\\Downloads\\Noise reduction 1\\output_denoised_image_folder",  # Specify the directory argument
    target_size=IMAGE_SIZE,
    batch_size=1,  # Use batch size 1 for single image prediction
    class_mode='categorical',
    shuffle=False  # Keep the order of predictions
)


# Get the number of samples in the test set
num_test_samples = test_generator.samples

# Get the predictions for the test set
y_pred = model.predict(test_generator, steps=num_test_samples)

# Get the ground truth labels for the test set
y_true = test_generator.classes

# Convert predictions to class labels
y_pred_classes = np.argmax(y_pred, axis=1)

# Get class labels from the generator
class_labels = list(test_generator.class_indices.keys())

# Convert class labels to numeric labels
numeric_labels = [test_generator.class_indices[label] for label in class_labels]

# Map numeric labels to class labels
numeric_to_label = dict(zip(numeric_labels, class_labels))

# Map numeric predictions to class labels
y_pred_labels = [numeric_to_label[label] for label in y_pred_classes]

# Convert ground truth labels to class labels
y_true_labels = [numeric_to_label[label] for label in y_true]

# Display the class labels and their numeric mappings
print("Class Labels:")
print(class_labels)
print("Numeric Labels:")
print(numeric_labels)

# Display the true and predicted labels for the test set
print("True Labels:")
print(y_true_labels)
print("Predicted Labels:")
print(y_pred_labels)

# Calculate the accuracy on the test set
test_acc = np.sum(y_pred_classes == y_true) / num_test_samples
print(f'Test Accuracy: {test_acc * 100}')


Found 136 images belonging to 17 classes.
Class Labels:
['0', '1', '10', '2', '3', '4', '5', '6', '7', '8', '9', 'box', 'fun', 'good', 'grow', 'hero', 'trust']
Numeric Labels:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
True Labels:
['0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '1', '1', '1', '1', '1', '1', '10', '10', '10', '10', '10', '10', '10', '10', '2', '2', '2', '2', '2', '2', '2', '2', '3', '3', '3', '3', '3', '3', '3', '3', '4', '4', '4', '4', '4', '4', '4', '4', '5', '5', '5', '5', '5', '5', '5', '5', '6', '6', '6', '6', '6', '6', '6', '6', '7', '7', '7', '7', '7', '7', '7', '7', '8', '8', '8', '8', '8', '8', '8', '8', '9', '9', '9', '9', '9', '9', '9', '9', 'box', 'box', 'box', 'box', 'box', 'box', 'box', 'box', 'fun', 'fun', 'fun', 'fun', 'fun', 'fun', 'fun', 'fun', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'grow', 'grow', 'grow', 'grow', 'grow', 'grow', 'grow', 'grow', 'hero', 'hero', 'hero', 'hero', 'hero', 'hero', 'hero', 'hero',

# VGG

In [11]:
from tensorflow.keras.models import load_model

model_paths = [
    #['D:\\FINAL MAJOR20-2\\ALL MODELS 2\\VGG\\VGG_kfold(1)_1.h5', 'D:\\FINAL MAJOR20-2\\ALL MODELS 2\\VGG\\VGG_kfold(1)_2.h5',
    # 'D:\\FINAL MAJOR20-2\\ALL MODELS 2\\VGG\\VGG_kfold(1)_3.h5', 'D:\\FINAL MAJOR20-2\\ALL MODELS 2\\VGG\\VGG_kfold(1)_4.h5',
    # 'D:\\FINAL MAJOR20-2\\ALL MODELS 2\\VGG\\VGG_kfold(1)_5.h5'],
    # Add paths for other models similarly
]

models = []
for model_fold_paths in model_paths:
    model_fold = []
    for model_path in model_fold_paths:
        model = load_model(model_path)
        model_fold.append(model)
    models.append(model_fold)
#model2 = load_model('D:\\FINAL MAJOR20-2\\VGG16\\vgg.h5')
#model3 = load_model('D:\\FINAL MAJOR20-2\\CNN_RNN\\cnn_rnn100.h5')
#model4 = load_model('D:\\FINAL MAJOR20-2\\resnet\\resnet.h5')
#model5 = load_model('D:\\FINAL MAJOR20-2\\ResnetMobile\\resnetMobile.h5')
#model6 = load_model('D:\\FINAL MAJOR20-2\\ResnetMobile_VGG\\resnetMobile_vgg_combined.h5')
#model7 = load_model('D:\\FINAL MAJOR20-2\\VGG-lstm\\vgg-lstm.h5')
#model8 = load_model('D:\\FINAL MAJOR20-2\\GLCM-LSTM\\kaggleDataset\\before_Kfold\\GLCM_LSTM.h5')

In [12]:

# Define constants
IMAGE_SIZE = (96, 72)
BATCH_SIZE = 16
EPOCHS = 100
NUM_CLASSES = 18
TRAIN_DATA_DIR = "C:\\Users\\sumit\\Downloads\\Noise reduction 1\\output_denoised_image_folder"

# Create a new data generator for the test data
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Generate the test data generator
test_generator = test_datagen.flow_from_directory(
    directory="C:\\Users\\sumit\\Downloads\\Noise reduction 1\\output_denoised_image_folder",  # Specify the directory argument
    target_size=IMAGE_SIZE,
    batch_size=1,  # Use batch size 1 for single image prediction
    class_mode='categorical',
    shuffle=False  # Keep the order of predictions
)


# Get the number of samples in the test set
num_test_samples = test_generator.samples

# Get the predictions for the test set
y_pred = model.predict(test_generator, steps=num_test_samples)

# Get the ground truth labels for the test set
y_true = test_generator.classes

# Convert predictions to class labels
y_pred_classes = np.argmax(y_pred, axis=1)

# Get class labels from the generator
class_labels = list(test_generator.class_indices.keys())

# Convert class labels to numeric labels
numeric_labels = [test_generator.class_indices[label] for label in class_labels]

# Map numeric labels to class labels
numeric_to_label = dict(zip(numeric_labels, class_labels))

# Map numeric predictions to class labels
y_pred_labels = [numeric_to_label[label] for label in y_pred_classes]

# Convert ground truth labels to class labels
y_true_labels = [numeric_to_label[label] for label in y_true]

# Display the class labels and their numeric mappings
print("Class Labels:")
print(class_labels)
print("Numeric Labels:")
print(numeric_labels)

# Display the true and predicted labels for the test set
print("True Labels:")
print(y_true_labels)
print("Predicted Labels:")
print(y_pred_labels)

# Calculate the accuracy on the test set
test_acc = np.sum(y_pred_classes == y_true) / num_test_samples
print(f'Test Accuracy: {test_acc * 100}')


Found 136 images belonging to 17 classes.
Class Labels:
['0', '1', '10', '2', '3', '4', '5', '6', '7', '8', '9', 'box', 'fun', 'good', 'grow', 'hero', 'trust']
Numeric Labels:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
True Labels:
['0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '1', '1', '1', '1', '1', '1', '10', '10', '10', '10', '10', '10', '10', '10', '2', '2', '2', '2', '2', '2', '2', '2', '3', '3', '3', '3', '3', '3', '3', '3', '4', '4', '4', '4', '4', '4', '4', '4', '5', '5', '5', '5', '5', '5', '5', '5', '6', '6', '6', '6', '6', '6', '6', '6', '7', '7', '7', '7', '7', '7', '7', '7', '8', '8', '8', '8', '8', '8', '8', '8', '9', '9', '9', '9', '9', '9', '9', '9', 'box', 'box', 'box', 'box', 'box', 'box', 'box', 'box', 'fun', 'fun', 'fun', 'fun', 'fun', 'fun', 'fun', 'fun', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'grow', 'grow', 'grow', 'grow', 'grow', 'grow', 'grow', 'grow', 'hero', 'hero', 'hero', 'hero', 'hero', 'hero', 'hero', 'hero',

# cnn_rnn

In [13]:
from tensorflow.keras.models import load_model

model_paths = [
    #['D:\\FINAL MAJOR20-2\\ALL MODELS\\CNN_RNN\\cnn-rnn_kfold(1)_1.h5', 'D:\\FINAL MAJOR20-2\\ALL MODELS\\CNN_RNN\\cnn-rnn_kfold(1)_2.h5',
    # 'D:\\FINAL MAJOR20-2\\ALL MODELS\\CNN_RNN\\cnn-rnn_kfold(1)_3.h5', 'D:\\FINAL MAJOR20-2\\ALL MODELS\\CNN_RNN\\cnn-rnn_kfold(1)_4.h5',
    # 'D:\\FINAL MAJOR20-2\\ALL MODELS\\CNN_RNN\\cnn-rnn_kfold(1)_4.h5'],
    # Add paths for other models similarly
]

models = []
for model_fold_paths in model_paths:
    model_fold = []
    for model_path in model_fold_paths:
        model = load_model(model_path)
        model_fold.append(model)
    models.append(model_fold)
#model2 = load_model('D:\\FINAL MAJOR20-2\\VGG16\\vgg.h5')
#model3 = load_model('D:\\FINAL MAJOR20-2\\CNN_RNN\\cnn_rnn100.h5')
#model4 = load_model('D:\\FINAL MAJOR20-2\\resnet\\resnet.h5')
#model5 = load_model('D:\\FINAL MAJOR20-2\\ResnetMobile\\resnetMobile.h5')
#model6 = load_model('D:\\FINAL MAJOR20-2\\ResnetMobile_VGG\\resnetMobile_vgg_combined.h5')
#model7 = load_model('D:\\FINAL MAJOR20-2\\VGG-lstm\\vgg-lstm.h5')
#model8 = load_model('D:\\FINAL MAJOR20-2\\GLCM-LSTM\\kaggleDataset\\before_Kfold\\GLCM_LSTM.h5')

In [14]:

# Define constants
IMAGE_SIZE = (96, 72)
BATCH_SIZE = 16
EPOCHS = 100
NUM_CLASSES = 18
TRAIN_DATA_DIR = "C:\\Users\\sumit\\Downloads\\Noise reduction 1\\output_denoised_image_folder"

# Create a new data generator for the test data
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Generate the test data generator
test_generator = test_datagen.flow_from_directory(
    directory="C:\\Users\\sumit\\Downloads\\Noise reduction 1\\output_denoised_image_folder",  # Specify the directory argument
    target_size=IMAGE_SIZE,
    batch_size=1,  # Use batch size 1 for single image prediction
    class_mode='categorical',
    shuffle=False  # Keep the order of predictions
)


# Get the number of samples in the test set
num_test_samples = test_generator.samples

# Get the predictions for the test set
y_pred = model.predict(test_generator, steps=num_test_samples)

# Get the ground truth labels for the test set
y_true = test_generator.classes

# Convert predictions to class labels
y_pred_classes = np.argmax(y_pred, axis=1)

# Get class labels from the generator
class_labels = list(test_generator.class_indices.keys())

# Convert class labels to numeric labels
numeric_labels = [test_generator.class_indices[label] for label in class_labels]

# Map numeric labels to class labels
numeric_to_label = dict(zip(numeric_labels, class_labels))

# Map numeric predictions to class labels
y_pred_labels = [numeric_to_label[label] for label in y_pred_classes]

# Convert ground truth labels to class labels
y_true_labels = [numeric_to_label[label] for label in y_true]

# Display the class labels and their numeric mappings
print("Class Labels:")
print(class_labels)
print("Numeric Labels:")
print(numeric_labels)

# Display the true and predicted labels for the test set
print("True Labels:")
print(y_true_labels)
print("Predicted Labels:")
print(y_pred_labels)

# Calculate the accuracy on the test set
test_acc = np.sum(y_pred_classes == y_true) / num_test_samples
print(f'Test Accuracy: {test_acc * 100}')


Found 136 images belonging to 17 classes.
Class Labels:
['0', '1', '10', '2', '3', '4', '5', '6', '7', '8', '9', 'box', 'fun', 'good', 'grow', 'hero', 'trust']
Numeric Labels:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
True Labels:
['0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '1', '1', '1', '1', '1', '1', '10', '10', '10', '10', '10', '10', '10', '10', '2', '2', '2', '2', '2', '2', '2', '2', '3', '3', '3', '3', '3', '3', '3', '3', '4', '4', '4', '4', '4', '4', '4', '4', '5', '5', '5', '5', '5', '5', '5', '5', '6', '6', '6', '6', '6', '6', '6', '6', '7', '7', '7', '7', '7', '7', '7', '7', '8', '8', '8', '8', '8', '8', '8', '8', '9', '9', '9', '9', '9', '9', '9', '9', 'box', 'box', 'box', 'box', 'box', 'box', 'box', 'box', 'fun', 'fun', 'fun', 'fun', 'fun', 'fun', 'fun', 'fun', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'grow', 'grow', 'grow', 'grow', 'grow', 'grow', 'grow', 'grow', 'hero', 'hero', 'hero', 'hero', 'hero', 'hero', 'hero', 'hero',

# vgg_lstm

In [15]:
from tensorflow.keras.models import load_model

model_paths = [
    ['D:\\FINAL MAJOR20-2\\ALL MODELS 2\\VGG_lstm\\VGG-lstm_kfold(1)_1.h5', 'D:\\FINAL MAJOR20-2\\ALL MODELS 2\\VGG_lstm\\VGG-lstm_kfold(1)_2.h5',
     'D:\\FINAL MAJOR20-2\\ALL MODELS 2\\VGG_lstm\\VGG-lstm_kfold(1)_3.h5', 'D:\\FINAL MAJOR20-2\\ALL MODELS 2\\VGG_lstm\\VGG-lstm_kfold(1)_4.h5',
     'D:\\FINAL MAJOR20-2\\ALL MODELS 2\\VGG_lstm\\VGG-lstm_kfold(1)_5.h5'],
    # Add paths for other models similarly
]

models = []
for model_fold_paths in model_paths:
    model_fold = []
    for model_path in model_fold_paths:
        model = load_model(model_path)
        model_fold.append(model)
    models.append(model_fold)
#model2 = load_model('D:\\FINAL MAJOR20-2\\VGG16\\vgg.h5')
#model3 = load_model('D:\\FINAL MAJOR20-2\\CNN_RNN\\cnn_rnn100.h5')
#model4 = load_model('D:\\FINAL MAJOR20-2\\resnet\\resnet.h5')
#model5 = load_model('D:\\FINAL MAJOR20-2\\ResnetMobile\\resnetMobile.h5')
#model6 = load_model('D:\\FINAL MAJOR20-2\\ResnetMobile_VGG\\resnetMobile_vgg_combined.h5')
#model7 = load_model('D:\\FINAL MAJOR20-2\\VGG-lstm\\vgg-lstm.h5')
#model8 = load_model('D:\\FINAL MAJOR20-2\\GLCM-LSTM\\kaggleDataset\\before_Kfold\\GLCM_LSTM.h5')

In [16]:

# Define constants
IMAGE_SIZE = (96, 72)
BATCH_SIZE = 16
EPOCHS = 100
NUM_CLASSES = 18
TRAIN_DATA_DIR = "C:\\Users\\sumit\\Downloads\\Noise reduction 1\\output_denoised_image_folder"

# Create a new data generator for the test data
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Generate the test data generator
test_generator = test_datagen.flow_from_directory(
    directory="C:\\Users\\sumit\\Downloads\\Noise reduction 1\\output_denoised_image_folder",  # Specify the directory argument
    target_size=IMAGE_SIZE,
    batch_size=1,  # Use batch size 1 for single image prediction
    class_mode='categorical',
    shuffle=False  # Keep the order of predictions
)


# Get the number of samples in the test set
num_test_samples = test_generator.samples

# Get the predictions for the test set
y_pred = model.predict(test_generator, steps=num_test_samples)

# Get the ground truth labels for the test set
y_true = test_generator.classes

# Convert predictions to class labels
y_pred_classes = np.argmax(y_pred, axis=1)

# Get class labels from the generator
class_labels = list(test_generator.class_indices.keys())

# Convert class labels to numeric labels
numeric_labels = [test_generator.class_indices[label] for label in class_labels]

# Map numeric labels to class labels
numeric_to_label = dict(zip(numeric_labels, class_labels))

# Map numeric predictions to class labels
y_pred_labels = [numeric_to_label[label] for label in y_pred_classes]

# Convert ground truth labels to class labels
y_true_labels = [numeric_to_label[label] for label in y_true]

# Display the class labels and their numeric mappings
print("Class Labels:")
print(class_labels)
print("Numeric Labels:")
print(numeric_labels)

# Display the true and predicted labels for the test set
print("True Labels:")
print(y_true_labels)
print("Predicted Labels:")
print(y_pred_labels)

# Calculate the accuracy on the test set
test_acc = np.sum(y_pred_classes == y_true) / num_test_samples
print(f'Test Accuracy: {test_acc * 100}')


Found 136 images belonging to 17 classes.
Class Labels:
['0', '1', '10', '2', '3', '4', '5', '6', '7', '8', '9', 'box', 'fun', 'good', 'grow', 'hero', 'trust']
Numeric Labels:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
True Labels:
['0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '1', '1', '1', '1', '1', '1', '10', '10', '10', '10', '10', '10', '10', '10', '2', '2', '2', '2', '2', '2', '2', '2', '3', '3', '3', '3', '3', '3', '3', '3', '4', '4', '4', '4', '4', '4', '4', '4', '5', '5', '5', '5', '5', '5', '5', '5', '6', '6', '6', '6', '6', '6', '6', '6', '7', '7', '7', '7', '7', '7', '7', '7', '8', '8', '8', '8', '8', '8', '8', '8', '9', '9', '9', '9', '9', '9', '9', '9', 'box', 'box', 'box', 'box', 'box', 'box', 'box', 'box', 'fun', 'fun', 'fun', 'fun', 'fun', 'fun', 'fun', 'fun', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'grow', 'grow', 'grow', 'grow', 'grow', 'grow', 'grow', 'grow', 'hero', 'hero', 'hero', 'hero', 'hero', 'hero', 'hero', 'hero',